# SIADS 516: Homework 3
Version 2.0.20201020.1
### Dr. Chris Teplovs, School of Information, University of Michigan
<small><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

This homework assignment builds on the Spark DataFrame material we covered in class.
You will be using a compressed version of the Yelp Academic Dataset.  The data set is provided for you in the data/yelp-academic sub-folder of this notebook's directory and you should not need to download it again if you're working on the Coursera hosted notebook environment.

You might want to refer to the lecture companion notebooks (in workspace-files/resources/lecture_notebooks or equivalently via Coursera as "Ungraded Lab: Spark Core Demo" and "Ungraded Lab: Spark SQL Demo) for hints about libraries to import, how to set up a SparkSession, and how to read data files.

You will notice that there are a **lot** of reviews.  You might want to work off a small sample (i.e. use the sample() function in Spark) to work on a reduced size dataset while you're developing your solution.

**You should take care to document your work, preferably using markdown blocks. In-code commenting is also 
a good idea.**

### <font color="magenta">Q1: How many users have received more than 5000 cool votes?</font>

In [1]:
#Create a SparkSession and SparkContext that will be used throughout the assignment
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('SIADS 516 Homework 3') \
    .getOrCreate() 

sc = spark.sparkContext

In [2]:
#Read in the Yelp Files
business = spark.read.json('data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('data/yelp_academic/yelp_academic_dataset_user.json.gz')

In [3]:
#Investigate the Dataframe
#user.printSchema()

#Similarly to a pandas boolean mask, the "User" Spark DataFrame was filtered to just 
#users with more than 5,000 complimnent_cool votes
user.filter(user['compliment_cool'] > 5000).count()

79

Using this filtering approach, it was found that 79 users have more than 5,000 compliment cool votes.

### <font color="magenta">Q2: What are the top 10 most useful positive reviews?</font>

In [4]:
#Investigate the Dataframe
#review.printSchema()

#Consider a Positive Review to be one with > 3 stars from the "Review" DataFrame with 3 stars being considered 
#a good middle/average score
positive_reviews = review.filter(review['stars'] > 3)

#From the "Positive Review" Spark Dataframe created above, a top 10 table of the most useful reviews is created first 
#by sorting descending based on the 'useful' column of the dataframe. To avoid a perhaps unnecessarily long output,
#just the review_id, business_id, stars, useful, and text of the review are returned. The Top 10 table is limited to the 
#first 10 reviews. These reviews are then collected and displayed below.
top_10 = positive_reviews.sort('useful', ascending = False).select(['review_id', 'business_id', 'stars', 'useful', 'text']).limit(10)
top_10.collect()

[Row(review_id='1lGXlyq4MALOMx17vpBcoQ', business_id='t-o_Sraneime4DDhWrQRBA', stars=5.0, useful=358, text='The Wynn Hotel. One word. Opulent. \n\nWynn, is a luxury resort and casino located on the Las Vegas Strip. Named after the casino developer Steve Wynn, this $2.7 billion resort together with Encore is the world\'s seventh-largest hotels and one of the finest in the world. \n\nHolding the title for the highest rated resort company in the world with 16 Forbes Travel Guide Five-Star Awards, AAA five diamond, Mobil five-star, Michelin five star, and Zagat Survey Top U.S. Hotel ratings, as well as one Michelin star for its restaurant Wing Lei, the first Chinese restaurant to receive a Michelin Star in the country. \n\nThe Wynn is a genre almost entirely their own. Talk about a blend of opulent creativity and seductive comfort. Wynn Hotel designer Roger Thomas calls it "Evocatect." The "architecture that evokes wonderful moments in people\'s lives." \n\nShare a moment alone or with som

As noted in the code above, "select" was used to limit to perhaps what could be considered the most important information (review_id, business_id, stars, useful, and text) of the positive reviews. With this information, you'd be able to easily search the Yelp datasets either for the review itself (with the review_id), the businesses with the most useful positive reviews (through business_id), etc. However, in terms of an interpretation, as seen above the most useful positive review (>3 stars) has been flagged as useful 358 times, with the 10th most useful review being flagged 215 times. 

### <font color="magenta">Q3: During what hour of the day do most checkins occur?</font>

In [5]:
#Investigate the Dataframe
#checkin.printSchema()
#checkin.take(2)

#Import pyspark functions udf (user-defined function), hour, and explode
from pyspark.sql.functions import udf, hour, explode
#Import the Array and String Types for use within a UDF
from pyspark.sql.types import ArrayType, StringType

#The datesplit UDF will look at the 'date' field in the checkin DataFrame, split on ', ' and return a list of strings, which
#are actually datetimes that can be utilized later
datesplit = udf(lambda x: x.split(', '), ArrayType(StringType()))

#From the checkin dataset, business_id is selected, the datesplit UDF is used to split out the string of checkin dates
#for each business and create a list of strings in the newly aliased 'dates' field. Using the explode function, a new
#column, checkin_time, is created from this 'dates' field that puts each timestamp for a business on an individual row.
exploded = checkin.select('business_id', datesplit('date').alias('dates')).withColumn('checkin_time', explode('dates'))

#With the timestamps now on an individual row, the hour function is used to pull the hour from the datetime, and a count
#of checkins are then grouped by this extracted hour. The count is sorted in descending order and this grouped by count
#is then shown.
exploded.groupBy(hour('checkin_time').alias('hour')).count().sort('count', ascending = False).show()

+----+-------+
|hour|  count|
+----+-------+
|   1|1561788|
|  19|1502271|
|   0|1491176|
|   2|1411255|
|  20|1350195|
|  23|1344117|
|  18|1272108|
|  22|1257437|
|  21|1238808|
|   3|1078939|
|  17|1006102|
|  16| 852076|
|   4| 747453|
|  15| 617830|
|   5| 485129|
|  14| 418340|
|   6| 321764|
|  13| 270145|
|   7| 231417|
|  12| 178910|
+----+-------+
only showing top 20 rows



Based on the output above, most checkins to businesses in the Yelp dataset occur at 1 am.

### <font color="magenta">Q4: Sentiment analysis</font>

a. List the 50 most common non-stopword words that are unique to *positive* reviews.
b. List the 50 most common non-stopword words that are unique to *negative* reviews.

You can use the stopword list that was introduced in the lecture materials or you can 
find/devise one of your own.

You will need to define what constitutes a positive review and what constitutes a negative review.  We highly recommend that you use the number of stars to figure this out.  Be sure to provide a rationale for your choice

As an example, consider the following two reviews:

* Positive: The meal was great, and the service was the best we ever experienced.
* Negative: The meal was awful.  It was the worst thing we ever experienced.

Assume our stopwords are {'the','was','and','the','was','we','it'}

* Positive unique: {'great', 'service', 'best'}

* Negative unique: {'awful', 'worst', 'thing'}

In this example, each unique word occurs just once, so the concept of "top 50" doesn't make sense.  For your data, you'll need to count the number of times each unique word occurs.

In [6]:
#Similarly to Q2, a positive review will be a review that has > 3 stars, while a negative review will be one with < 3 stars.
#This is again with the assumption that a 3 star review is a strong indicator of average and anything to the right or left
#of 3 would be considered positive and negative, respectively.

#Import library to allow for some regex functions to be utilized
import re

#Stop words were pulled from the Week 1 mrjob assignment
stopwords = ['i', 'we', 'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than']

#A UDF was created below that found all words
## 'r' specifies that the '\' in the regex should not be treated as an escape
## '\b' are used as the boundary for a word and will prevent whitespace and punctuation from being included
## '\S+' will find one or more instances of characters for words-only

#This regex in the UDF will parse the lower case characters in the string it's called on and the UDF itself
#will return a list of strings
word_find = udf(lambda x: re.findall(r'\b\S+\b', x.lower()), ArrayType(StringType()))

#Positive reviews are again considered a review with > 3 stars and a negative review < 3 stars. Only the 'text'
#column from the review DataFrame is selected following the star filter
positive = review.filter(review['stars'] > 3).select('text')
negative = review.filter(review['stars'] < 3).select('text')

#To create positive1, the 'text' field is selected from the positive review DF created above, but an additional field
#aliased/named 'list' is also created when utilizing the prior created word_find UDF on the 'text' field. Lastly, a new
#column 'value' is created when exploding this 'list' field, which now puts every word found in the positive reviews on its
#own line.
positive1 = positive.select(['text', word_find('text').alias('list')]).withColumn('value', explode('list'))
#Only words from the positive reviews that are NOT (~) in the above stopwords list are selected from the 'value' column
#following the filter
positive_final = positive1.filter(~positive1.value.isin(stopwords)).select('value')

#The same approach as noted above for the positive reviews is applied to the negative review DF as well to generate
#the two below DataFrames
negative1 = negative.select(['text', word_find('text').alias('list')]).withColumn('value', explode('list'))
negative_final = negative1.filter(~negative1.value.isin(stopwords)).select('value')

#As a means of reducing the size of the DataFrames, the words (value) are counted in a groupby for both the positive
#and negative reviews
positive_count = positive_final.groupBy('value').count()
negative_count = negative_final.groupBy('value').count()

#To generate a collection of positive review words that are unique from the negative review words and vice versa,
#the grouped-by dataframes are left anti joined, which will return the values from the left table that are not found
#in the right. Left Anti Joining the positive reviews to negative and vice versa results in DataFrames that have
#unique words between the two.
positive_clean = positive_count.join(negative_count, on = 'value', how = 'leftanti')
negative_clean = negative_count.join(positive_count, on = 'value', how = 'leftanti')



In [7]:
#The positive review words are sorted by descending based on count and the top 50 is displayed.
positive_clean.sort('count', ascending = False).show(50)

+---------------+-----+
|          value|count|
+---------------+-----+
|          eloff|  291|
|           jabz|  202|
|         fixler|  157|
|    ahhhhmazing|  156|
|      heartwood|  139|
|         popbar|  138|
|            ceg|  138|
|         boothe|  127|
|           emme|  127|
|          artur|  123|
|            f45|  117|
|          safak|  115|
|      delizioso|  112|
|   shutterbooth|  108|
|         sidell|  106|
|      try...you|  104|
|           homa|   99|
|            imr|   99|
|       perfects|   97|
|         exquis|   96|
|           jayd|   93|
|    dee-licious|   91|
|      ahhmazing|   91|
|      merveille|   90|
|        koshari|   90|
|        hobgood|   90|
|    ah-maze-ing|   87|
|        crémeux|   87|
|          trego|   86|
|         omalza|   85|
|  scrumptiously|   83|
|  place...great|   82|
|           insp|   80|
|         hiroba|   79|
|          saura|   79|
|    ahhh-mazing|   79|
|          hubba|   79|
|        kikuchi|   78|
|           etti

In [8]:
#The negative review words are sorted by descending based on count and the top 50 is displayed.
negative_clean.sort('count', ascending = False).show(50)

+-------------+-----+
|        value|count|
+-------------+-----+
|   neveragain|   72|
| unsubscribed|   53|
|      no-star|   42|
| unempathetic|   38|
|   con-artist|   33|
|       pesimo|   30|
|       voiers|   30|
|         fahw|   30|
|    discusted|   30|
|  un-returned|   29|
|   dealticker|   29|
|       fakest|   28|
|  coinsurance|   28|
|          rmi|   28|
|        gager|   28|
|   unlawfully|   28|
|       sorrys|   27|
|   nonpayment|   27|
|  ex-customer|   27|
|          amj|   26|
|     ontrac's|   26|
| irish/jewish|   26|
|       theifs|   26|
|          ybt|   25|
|     horibble|   24|
|    exécrable|   23|
|   magistrate|   23|
|         cfpb|   23|
|   transunion|   23|
|  voicemail's|   23|
|          hra|   23|
|         tkps|   22|
|     comenity|   22|
|        2.5'd|   22|
|      horable|   21|
|        10:12|   20|
|     rude.the|   20|
|unsubscribing|   20|
|      suppost|   20|
|        nedia|   20|
| legionnaires|   20|
|        11:13|   20|
|    8/14/

The top 50 words from positive and negative reviews align with expectations. In the positive reviews, you see frequent words such as "oh-so-delicious", "ahhhhmazing", "scrumptiously", etc. whereas in the negative reviews "fraudster", "neveragain", "unsubscribed", "no-star" are commonly found. These top 50 lists both align well with what you would expect to see alongside >3 star or < 3 star reviews.